In [197]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [198]:
import os
print(os.getcwd())

C:\Users\guswh\Desktop\data-analysis\dacon_housePrediction\src


In [199]:
train = pd.read_csv('../data/prep/train3.csv')
test = pd.read_csv('../data/prep/test3.csv')
submission = pd.read_csv('../data/raw/housing/sample_submission.csv')

In [200]:
train

,id,Gr.Liv.Area,Garage.Cars,Garage.Area,Total.Bsmt.SF,X1st.Flr.SF,Full.Bath,Year.Built,Garage.Yr.Blt_cal,Year.Remod.Add_cal,Year.Built_cal,comb.Built,total.Price.Index,target
0,1,7.780303,1.386294,6.876265,7.780303,7.780303,1.098612,7.602900,2.197225,2.197225,2.197225,2.833213,14.015673,12.864243
1,2,7.210080,1.098612,6.146329,7.210080,7.210080,1.098612,7.604396,1.791759,1.609438,1.791759,2.302585,12.927572,12.175619
2,3,6.803505,0.693147,5.666427,6.762730,6.803505,0.693147,7.584773,3.806662,3.806662,3.806662,4.488636,11.796908,11.719948
3,4,7.069023,1.098612,6.357842,6.523562,6.523562,0.693147,7.550135,2.484907,1.791759,4.718499,4.762174,11.954407,11.813037
4,5,7.580189,1.386294,6.842683,6.934397,6.934397,1.098612,7.603898,1.945910,1.945910,1.945910,2.564949,12.937154,12.429220
5,6,7.585281,1.386294,6.523562,6.652863,6.652863,1.098612,7.605890,1.098612,0.693147,1.098612,1.386294,13.060388,12.504327
6,7,7.299121,1.098612,6.093570,7.299121,7.299121,0.693147,7.579679,4.007333,4.007333,4.007333,4.691348,12.481068,11.960818
7,8,7.833996,1.098612,6.297109,7.833996,7.833996,1.098612,7.591862,3.433987,3.433987,3.433987,4.110874,13.565696,12.535380
8,9,7.882315,1.386294,6.616065,7.299797,7.323831,1.098612,7.601902,2.397895,2.302585,2.397895,2.995732,13.406725,12.950984
9,10,7.273093,1.098612,6.148468,7.267525,7.273093,1.098612,7.602900,2.197225,2.197225,2.197225,2.833213,13.116380,12.356650


In [201]:
train = train.iloc[:, 1:]
test = test.iloc[:, 1:]

In [202]:
X = train.drop('target', axis = 1)
#y = np.log1p(train.target)
y = train.target
target = test[X.columns]

In [203]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold

In [204]:
def NMAE(true, pred) -> float:
    mae = np.mean(np.abs(true - pred))
    score = mae / np.mean(np.abs(true))
    return score

In [205]:
nmae_score = make_scorer(NMAE, greater_is_better=False)

In [206]:
kf = KFold(n_splits = 10, random_state = 42, shuffle = True)

In [207]:
cb_pred = np.zeros(target.shape[0])
cb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    tr_data = Pool(data = tr_x, label = tr_y)
    val_data = Pool(data = val_x, label = val_y)
    
    cb = CatBoostRegressor(depth = 4, random_state = 42, loss_function = 'MAE', n_estimators = 3000, learning_rate = 0.03, verbose = 0)
    cb.fit(tr_data, eval_set = val_data, early_stopping_rounds = 750, verbose = 1000)
    
    val_pred = np.expm1(cb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    cb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = cb.predict(target) / 10
    cb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(cb_val)} & std = {np.std(cb_val)}')

1 FOLD Training.....
0:	learn: 0.2876652	test: 196772.7100662	best: 196772.7100662 (0)	total: 1.38ms	remaining: 4.13s
1000:	learn: 0.0677679	test: 196772.6299075	best: 196772.6271750 (385)	total: 1.65s	remaining: 3.3s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 196772.6272
bestIteration = 385

Shrink model to first 386 iterations.
1 FOLD NMAE = 0.08866730753982856

2 FOLD Training.....
0:	learn: 0.2898049	test: 186667.9097102	best: 186667.9097102 (0)	total: 4.46ms	remaining: 13.4s
1000:	learn: 0.0668231	test: 186667.8472383	best: 186667.8472144 (997)	total: 2.17s	remaining: 4.34s
2000:	learn: 0.0594200	test: 186667.8466225	best: 186667.8465563 (1964)	total: 4.49s	remaining: 2.24s
2999:	learn: 0.0554715	test: 186667.8460000	best: 186667.8460000 (2999)	total: 6.59s	remaining: 0us

bestTest = 186667.846
bestIteration = 2999

2 FOLD NMAE = 0.09809120115485294

3 FOLD Training.....
0:	learn: 0.2877842	test: 192062.6953870	best: 192062.6953870 (0)	total: 2.22ms	remaini

In [ ]:
# ngboost
ngb_pred = np.zeros(target.shape[0])
ngb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    ngb = NGBRegressor(random_state = 42, n_estimators = 1000, verbose = 0, learning_rate = 0.03)
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 300)
    
    val_pred = np.expm1(ngb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    ngb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = ngb.predict(target) / 10
    ngb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(ngb_val)} & std = {np.std(ngb_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.0853459372604042

2 FOLD Training.....
2 FOLD NMAE = 0.10360249741300927

3 FOLD Training.....
3 FOLD NMAE = 0.0949285014276525

4 FOLD Training.....
4 FOLD NMAE = 0.09559228472914556

5 FOLD Training.....
5 FOLD NMAE = 0.08175724991781823

6 FOLD Training.....
6 FOLD NMAE = 0.09812931278952616

7 FOLD Training.....
7 FOLD NMAE = 0.09241054403978206

8 FOLD Training.....
8 FOLD NMAE = 0.09750066134429751

9 FOLD Training.....


In [ ]:
submission['target'] = ngb_pred

In [ ]:
submission

In [ ]:
submission.to_csv("../out/ngb/ngb1.csv", header = True, index = False)